<a href="https://colab.research.google.com/github/Dowell-Lab/pop_inf_beta/blob/main/colab/genes_are_different.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
import plotly.express as px

In [2]:
indir="https://raw.githubusercontent.com/Dowell-Lab/pop_inf_beta/refs/heads/main/testdata/"


In [3]:
metadataPRO = "metaPRO.txt"
metadataRNA = "metaRNA.txt"
popdiffPRO = "pop_PROgenebody_PROseq_res.txt"
popdiffRNA = "pop_RNAgene_RNAseq_res.txt"
normcountsPRO = "normalized__master__PRO.csv"
normcountsRNA = "normalized__master__RNA.csv"
PROsufix= "_PROgenebody_PROseq_res.txt"
RNAsufix= "_RNAgene_RNAseq_res.txt"

# Pull in the meta data for the experiment

In [4]:
fn = indir+metadataPRO
PROmeta = pd.read_csv(fn)
fn = indir+metadataRNA
RNAmeta = pd.read_csv(fn)

In [5]:
people = PROmeta["genotype"].unique()
people

array(['ChenChao', 'Dave', 'Eric', 'Ethan', 'Khaondo', 'Niyilolawa',
       'Pedro', 'Srivathani'], dtype=object)

# Pull in the normalized counts for every gene in every person

In [6]:
PROcounts = pd.read_csv(indir+normcountsPRO, index_col=0)
RNAcounts = pd.read_csv(indir+normcountsRNA, index_col=0)

# Which genes are different in the population after treating the population with interferon beta. This was measured via both PRO-seq and RNA-seq.

In [7]:
popdiffPROdf = pd.read_csv(indir+popdiffPRO, index_col=0)
popdiffPROdf = popdiffPROdf.sort_values("padj")
popdiffRNAdf = pd.read_csv(indir+popdiffRNA, index_col=0)
popdiffRNAdf = popdiffRNAdf.sort_values("padj")

In [8]:
popdiffRNAdf

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
DTX3L,11268.033605,3.254709,0.107344,30.320488,6.156466e-202,1.412540e-197
USP18,3161.533571,4.362786,0.148979,29.284542,1.631945e-188,1.872167e-184
SLFN5,12528.305968,3.220319,0.110883,29.042399,1.919481e-185,1.468019e-181
APOL6,15036.688842,3.437850,0.118853,28.925306,5.739085e-184,2.633551e-180
PLSCR1,3834.971435,3.494580,0.120801,28.928439,5.241196e-184,2.633551e-180
...,...,...,...,...,...,...
SPRY3_1,0.000000,NaN,NaN,NaN,NaN,NaN
VAMP7_1,0.000000,NaN,NaN,NaN,NaN,NaN
IL9R_1,0.000000,NaN,NaN,NaN,NaN,NaN
WASIR1_1,0.000000,NaN,NaN,NaN,NaN,NaN


# Find the adjusted p-value for each gene in each person and each assay

In [9]:
padj_per_person = pd.DataFrame(index=popdiffRNAdf.index)
padj_per_person["pop_RNA"] = popdiffRNAdf["padj"]
padj_per_person["pop_PRO"] = popdiffPROdf["padj"]

for person in people:
    print(person)
    persondiffPROdf = pd.read_csv(indir+person+PROsufix, index_col=0)
    persondiffRNAdf = pd.read_csv(indir+person+RNAsufix, index_col=0)
    persondiffPROdf = persondiffPROdf[["padj"]]
    persondiffRNAdf = persondiffRNAdf[["padj"]]
    persondiffPROdf.columns = [person+"_PRO"]
    persondiffRNAdf.columns = [person+"_RNA"]
    padj_per_person = pd.concat([padj_per_person, persondiffPROdf, persondiffRNAdf], axis=1)




ChenChao
Dave
Eric
Ethan
Khaondo
Niyilolawa
Pedro
Srivathani


# Create a data frame with only genes that significant in at least one comparison (In one person or in the population in RNA seq or in PRO-seq)

In [10]:
threshold=0.01
sig_padj_per_person = padj_per_person.copy()
sig_padj_per_person["min_padj"] = sig_padj_per_person.min(axis=1)
sig_padj_per_person = sig_padj_per_person[sig_padj_per_person["min_padj"]<threshold] #If it was significant in any experiment keep the gene
sig_padj_per_person.sort_values("pop_RNA")

,pop_RNA,pop_PRO,ChenChao_PRO,ChenChao_RNA,Dave_PRO,Dave_RNA,Eric_PRO,Eric_RNA,Ethan_PRO,Ethan_RNA,Khaondo_PRO,Khaondo_RNA,Niyilolawa_PRO,Niyilolawa_RNA,Pedro_PRO,Pedro_RNA,Srivathani_PRO,Srivathani_RNA,min_padj
DTX3L,1.412540e-197,4.360128e-38,4.360128e-38,1.412540e-197,8.583627e-23,2.587407e-101,5.030985e-40,4.767178e-268,2.992660e-40,4.880995e-209,7.572255e-32,5.633599e-147,4.193822e-42,2.036026e-172,4.756796e-29,3.995864e-146,1.883170e-34,1.222962e-157,4.767178e-268
USP18,1.872167e-184,1.459065e-49,1.459065e-49,1.872167e-184,9.385243e-24,1.964340e-118,1.788279e-36,8.571460e-183,4.936345e-38,3.440160e-153,5.891097e-45,3.310760e-158,9.124827e-51,1.337141e-151,1.405153e-54,4.613727e-204,2.219790e-34,1.107774e-130,4.613727e-204
SLFN5,1.468019e-181,3.964858e-40,3.964858e-40,1.468019e-181,9.834914e-33,8.481532e-160,1.833730e-45,1.594368e-248,7.212161e-48,2.358637e-245,3.448248e-39,3.053801e-146,7.942320e-51,5.944580e-198,1.275917e-51,3.865349e-190,6.941539e-39,4.603887e-140,1.594368e-248
APOL6,2.633551e-180,5.671051e-59,5.671051e-59,2.633551e-180,4.336187e-30,1.981444e-76,2.624352e-38,5.449274e-150,6.167975e-61,5.945410e-205,2.149333e-68,8.888193e-184,2.688942e-61,1.228000e-155,2.663202e-60,2.117945e-166,1.498494e-46,6.630205e-149,5.945410e-205
PLSCR1,2.633551e-180,1.717607e-47,1.717607e-47,2.633551e-180,1.238461e-46,3.088674e-154,3.160305e-63,4.538355e-259,7.128719e-68,2.078463e-241,2.170758e-38,3.081114e-111,1.389872e-55,6.054947e-192,2.624259e-54,9.887244e-206,4.472556e-44,4.491567e-145,4.538355e-259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LOC105376265,NaN,2.007870e-04,2.007870e-04,NaN,6.742309e-01,NaN,5.931764e-01,NaN,8.342790e-01,6.122152e-01,4.293560e-01,7.672554e-01,4.917870e-01,9.981534e-01,6.917063e-01,NaN,9.354765e-01,NaN,2.007870e-04
LOC613206,NaN,1.927038e-01,1.927038e-01,NaN,1.661774e-01,NaN,2.189470e-01,NaN,2.149226e-01,NaN,2.802404e-03,NaN,3.562111e-02,NaN,8.582688e-01,NaN,9.397871e-01,NaN,2.802404e-03
LOC124902295,NaN,6.924051e-02,6.924051e-02,NaN,1.055255e-01,NaN,5.728212e-02,NaN,1.318091e-02,7.982670e-01,1.371913e-01,1.262902e-01,2.161579e-03,9.769784e-01,6.384940e-02,NaN,2.274832e-01,NaN,2.161579e-03
LOC105376314,NaN,1.077472e-02,1.077472e-02,NaN,6.619048e-02,NaN,9.118980e-01,NaN,5.859693e-03,NaN,2.907501e-02,NaN,3.398457e-02,NaN,7.813282e-02,NaN,3.742192e-01,NaN,5.859693e-03


# Find genes that are significant in the population but not in any one person

In [11]:
keepcols = [cn for cn in sig_padj_per_person.columns if "pop" in cn]
pop_adj_df = sig_padj_per_person[keepcols]
keepcols = [cn for cn in sig_padj_per_person.columns if not "pop" in cn]
people_adj_df = sig_padj_per_person[keepcols]
keepcols = [cn for cn in people_adj_df.columns if "RNA" in cn]
people_adj_df_RNA = people_adj_df[keepcols]
keepcols = [cn for cn in people_adj_df.columns if "PRO" in cn]
people_adj_df_PRO = people_adj_df[keepcols]
keepcols = [cn for cn in pop_adj_df.columns if "RNA" in cn]
pop_adj_df_RNA = pop_adj_df[keepcols]
keepcols = [cn for cn in pop_adj_df.columns if "PRO" in cn]
pop_adj_df_PRO = pop_adj_df[keepcols]

In [12]:
sig_pop_RNA = pop_adj_df_RNA[pop_adj_df_RNA["pop_RNA"]<threshold]
sig_pop_PRO = pop_adj_df_PRO[pop_adj_df_PRO["pop_PRO"]<threshold]
sig_people_RNA = people_adj_df_RNA[people_adj_df_RNA.min(axis=1)<threshold]
sig_people_PRO = people_adj_df_PRO[people_adj_df_PRO.min(axis=1)<threshold]
sig_all_people_RNA = people_adj_df_RNA[people_adj_df_RNA.max(axis=1)<threshold]
sig_all_people_PRO = people_adj_df_PRO[people_adj_df_PRO.max(axis=1)<threshold]

In [13]:
pop_adj_df_RNA

,pop_RNA
DTX3L,1.412540e-197
USP18,1.872167e-184
SLFN5,1.468019e-181
APOL6,2.633551e-180
PLSCR1,2.633551e-180
...,...
LOC105376265,NaN
LOC613206,NaN
LOC124902295,NaN
LOC105376314,NaN


In [14]:
#people_adj_df_RNA.to_csv("/Users/allenma/pop_inf_beta/testdata/pop_adj_df_RNA.csv")

In [15]:
#people_adj_df_PRO.to_csv("/Users/allenma/pop_inf_beta/testdata/pop_adj_df_PRO.csv")

In [16]:
num_above_threshold = pd.DataFrame(index=people_adj_df_RNA.index)

# Calculate the number of columns above the threshold for each row
num_above_threshold['num_above_threshold_RNA'] = (people_adj_df_RNA > threshold).sum(axis=1)

# Calculate the number of columns above the threshold for each row
num_above_threshold['num_above_threshold_PRO'] = (people_adj_df_PRO > threshold).sum(axis=1)


In [17]:

RNA_above_threshold_cols = people_adj_df_RNA[people_adj_df_RNA > threshold].notna()
PRO_above_threshold_cols = people_adj_df_PRO[people_adj_df_PRO > threshold].notna()

In [18]:
RNA_genes_pop_not_people = [gn for gn in sig_pop_RNA.index if gn not in sig_people_RNA.index]
print("RNA genes", RNA_genes_pop_not_people)
PRO_genes_pop_not_people = [gn for gn in sig_pop_PRO.index if gn not in sig_people_PRO.index]
print("PRO genes", PRO_genes_pop_not_people)

RNA genes []
PRO genes []


# Find genes that are significant in people not in population

In [19]:
RNA_genes_pop_not_people = [gn for gn in sig_people_RNA.index if gn not in sig_pop_RNA.index]
print("RNA genes", RNA_genes_pop_not_people)
PRO_genes_pop_not_people = [gn for gn in sig_people_PRO.index if gn not in sig_pop_PRO.index]
print("PRO genes", PRO_genes_pop_not_people)



RNA genes ['CEMIP2', 'REXO4', 'ANKRD34A', 'POM121', 'FAM3C', 'TMOD3', 'IMP3', 'JMJD7-PLA2G4B', 'PSD4', 'KHK', 'LRRC27', 'DVL2', 'SLC20A1', 'PALS2', 'KRBA2', 'MIR320E', 'WASL', 'RHOG', 'TMC8', 'KIAA0408', 'LINC02390', 'TESPA1', 'PREX1', 'KLHL24', 'ITPR3', 'TBC1D13', 'STX7', 'ARHGAP11B', 'CNTROB', 'CECR7', 'IL18RAP', 'SOGA3', 'CCL5', 'NSG1', 'MRPS26', 'FCRLA', 'POLR3H', 'SOX12', 'TNKS', 'RSAD1', 'USP51', 'WBP1L', 'MAP7D1', 'CAMK2G', 'SS18', 'C1orf35', 'RAB14', 'ITPKC', 'SNTB2', 'ACTR8', 'UBASH3B', 'TYSND1', 'NETO1', 'LRRC20', 'SLC19A2', 'OTULIN-DT', 'LASP1', 'ARSK', 'PPAN-P2RY11', 'RTL6', 'HS6ST1', 'SOWAHB', 'ANKS3', 'CNPY3', 'PLEKHN1', 'LOC100130987', 'CXCL11', 'DUSP14', 'LOC107985289', 'ITGAV', 'EFNA4', 'ZNF785', 'NCLN', 'PMEPA1', 'LOC105377626', 'SELL', 'PRKX', 'HEATR5B', 'GFER', 'GUCY1B1', 'GDPD3', 'CEACAM21', 'GNAZ', 'MAN2A1', 'SLFN14', 'QPRT', 'AGRN', 'TGFB1', 'ERCC6L2', 'LINC00294', 'CLN3', 'LINC00539', 'FCRL5', 'CYFIP2', 'TNIP1', 'MAPK8IP3', 'FKBP5', 'KCNQ1', 'VEGFB', 'EVI5L', 'R

# Find gene sig in some people but not all

In [20]:
RNA_genes_pop_not_people = [gn for gn in sig_people_RNA.index if gn not in sig_all_people_RNA.index]
print("RNA genes", RNA_genes_pop_not_people)
PRO_genes_pop_not_people = [gn for gn in sig_people_PRO.index if gn not in sig_all_people_PRO.index]
print("PRO genes", PRO_genes_pop_not_people)


RNA genes ['TSPAN5', 'TMEM255A', 'DSP', 'SYN2', 'GFI1', 'KIAA0513', 'L3MBTL2-AS1', 'BCL11A', 'CACNA1D', 'ARHGAP25', 'NOD2', 'SLA2', 'FAM53A', 'GRM4', 'GATA3', 'CYP2J2', 'NOD1', 'DIPK1A', 'FAM53B', 'ATP10A', 'STOM', 'DNMBP', 'POU2F2', 'SLC15A4', 'CRTC3', 'ZBTB20', 'SIGLEC14', 'FAM13A', 'PRKD2', 'HTR4', 'LLGL2', 'TMEM38A', 'LOC105377327', 'CAPN2', 'NXPE3', 'MAB21L3', 'IGF1', 'NAA25', 'TBC1D14', 'FBXW7', 'RNF144B', 'LPAR5', 'LOC105374764', 'CSF1', 'DENND5A', 'CCDC134', 'LOC105379358', 'SCML4', 'MDGA1', 'CRYBG1', 'LINC02964', 'CALHM2', 'TREML2', 'LINC00537', 'PPP1R12B', 'RUBCN', 'TCP11L1', 'BCL9', 'PAX3', 'N4BP2L1', 'LOC105374843', 'C11orf54', 'CPNE5', 'SECTM1', 'PALM2AKAP2', 'NOTCH1', 'ARL5B', 'LOC124904244', 'CBLB', 'SMAD1', 'FMNL2', 'CD180', 'ATXN1L', 'GLUL', 'INAVA', 'LINC01260', 'CASK', 'CDCP1', 'KIAA1614', 'RAB11FIP4', 'SYCP2L', 'LOC100506639', 'ARHGEF38', 'TP53INP2', 'MYO5C', 'GIMAP8', 'TEX29', 'LOC105373404', 'MYL12BP2', 'LOC105371692', 'TRPC4AP', 'TMC3-AS1', 'RIPOR1', 'PDZD2', 'PA

# Plot one gene

In [42]:

# Reshape the PROcounts DataFrame from wide to long format
PROcounts_long = PROcounts.reset_index().melt(id_vars='index', var_name='sample', value_name='count')

# Split the sample column into 'name' and 'assay' columns
PROcounts_long[['assay', 'treatment',"person", "replicate"]] = PROcounts_long['sample'].str.split('.', expand=True)

# Reshape the RNAcounts DataFrame from wide to long format
RNAcounts_long = RNAcounts.reset_index().melt(id_vars='index', var_name='sample', value_name='count')

# Split the sample column into 'name' and 'assay' columns
RNAcounts_long[['assay', 'treatment',"person", "replicate"]] = RNAcounts_long['sample'].str.split('.', expand=True)



In [43]:
def getonegene(onegene="STMN1"):
  one_gene_PROcounts_long = PROcounts_long[PROcounts_long["index"]==onegene]
  one_gene_RNAcounts_long = RNAcounts_long[RNAcounts_long["index"]==onegene]
  one_gene_long = pd.concat([one_gene_PROcounts_long, one_gene_RNAcounts_long])
  return one_gene_long

  # prompt: make two new columns in one_gene_long that come from the treatment column and the values are the count column

def one_gene_long_to_IFN_BSA_split(one_gene_long):
# Create new columns 'treatment_count' and 'treatment_count'
  one_gene_long['experiment']= one_gene_long['assay'] + '.' + one_gene_long['person'] + '.' + one_gene_long['replicate']
  one_gene_long_BSA = one_gene_long[one_gene_long['treatment'].str.contains('BSA')]
  one_gene_long_IFN = one_gene_long[one_gene_long['treatment'].str.contains('IFN')]
  one_gene_long_BSA = one_gene_long_BSA[["experiment", "assay", "person", "replicate", "count"]]
  one_gene_long_IFN = one_gene_long_IFN[["experiment", "count"]]
  combo = one_gene_long_BSA.merge(one_gene_long_IFN, on="experiment", suffixes=("_BSA", "_IFN"))
  return combo

## Pick one gene

In [51]:
onegene="STMN1"
#onegene="TSPAN5"
#onegene="TMEM255A"

one_gene_long = getonegene(onegene=onegene)


In [52]:

is_sig_pop_RNA = [onegene in sig_pop_RNA.index]
is_sig_pop_PRO = [onegene in sig_pop_PRO.index]
is_sig_anyone_RNA = [onegene in sig_people_RNA.index]
is_sig_anyone_PRO = [onegene in sig_people_PRO.index]
is_sig_all_RNA = [onegene in sig_all_people_RNA.index]
is_sig_all_PRO = [onegene in sig_all_people_PRO.index]


print(onegene)
print("is_sig_pop_RNA", is_sig_pop_RNA[0])
print("is_sig_pop_PRO", is_sig_pop_PRO[0])
print("is_sig_anyone_RNA", is_sig_anyone_RNA[0])
print("is_sig_anyone_PRO", is_sig_anyone_PRO[0])
print("is_sig_all_RNA", is_sig_all_RNA[0])
print("is_sig_all_PRO", is_sig_all_PRO[0])
print(num_above_threshold[num_above_threshold.index==onegene])

STMN1
is_sig_pop_RNA False
is_sig_pop_PRO False
is_sig_anyone_RNA False
is_sig_anyone_PRO True
is_sig_all_RNA False
is_sig_all_PRO False
       num_above_threshold_RNA  num_above_threshold_PRO
STMN1                        8                        7


In [53]:

whichpeopleRNA = RNA_above_threshold_cols.loc[onegene].to_frame()
whichpeopleRNA = whichpeopleRNA[whichpeopleRNA[onegene]==True]
print("Sig in RNA for ", sorted(whichpeopleRNA.index))

whichpeoplePRO = PRO_above_threshold_cols.loc[onegene].to_frame()
whichpeoplePRO = whichpeoplePRO[whichpeoplePRO[onegene]==True]
print("Sig in PRO for ", sorted(whichpeoplePRO.index))

Sig in RNA for  ['ChenChao_RNA', 'Dave_RNA', 'Eric_RNA', 'Ethan_RNA', 'Khaondo_RNA', 'Niyilolawa_RNA', 'Pedro_RNA', 'Srivathani_RNA']
Sig in PRO for  ['ChenChao_PRO', 'Dave_PRO', 'Eric_PRO', 'Ethan_PRO', 'Niyilolawa_PRO', 'Pedro_PRO', 'Srivathani_PRO']


In [54]:
one_gene_long

,index,sample,count,assay,treatment,person,replicate
665,STMN1,PRO.BSA.ChenChao.1,1532.844714,PRO,BSA,ChenChao,1
42908,STMN1,PRO.BSA.ChenChao.2,1415.304513,PRO,BSA,ChenChao,2
85151,STMN1,PRO.BSA.Dave.1,815.451741,PRO,BSA,Dave,1
127394,STMN1,PRO.BSA.Dave.3,1118.321847,PRO,BSA,Dave,3
169637,STMN1,PRO.BSA.Eric.1,1970.331842,PRO,BSA,Eric,1
...,...,...,...,...,...,...,...
1817114,STMN1,RNA.IFN.Pedro.2,7625.063336,RNA,IFN,Pedro,2
1859357,STMN1,RNA.IFN.Pedro.3,7627.898256,RNA,IFN,Pedro,3
1901600,STMN1,RNA.IFN.Srivathani.1,6140.611613,RNA,IFN,Srivathani,1
1943843,STMN1,RNA.IFN.Srivathani.2,6718.818481,RNA,IFN,Srivathani,2


In [63]:

def plot_one_gene_scatter(one_gene_long):
    fig = px.scatter(one_gene_long, x="person", y="count", facet_row="assay", color="treatment",symbol="replicate",
                     category_orders={"treatment": ["BSA", "IFN"], "replicate":[1,2,3]})
    fig.show()



In [65]:
plot_one_gene_scatter(one_gene_long)


In [26]:
one_gene_long = getonegene(onegene=onegene)
onegene_BSA_IFN = one_gene_long_to_IFN_BSA_split(one_gene_long)
onegene_BSA_IFN_RNA = onegene_BSA_IFN[onegene_BSA_IFN["assay"]=="RNA"]
onegene_BSA_IFN_PRO = onegene_BSA_IFN[onegene_BSA_IFN["assay"]=="PRO"]

In [27]:
fig = px.scatter(onegene_BSA_IFN_RNA, x="count_BSA", y="count_IFN", trendline="ols", color="person")

fig.show()

In [28]:
fig = px.scatter(onegene_BSA_IFN_RNA, x="count_BSA", y="count_IFN", trendline="ols", color="person")


# Force the x and y axes to have the same range
axis_range = [min(min(onegene_BSA_IFN_RNA['count_BSA']), min(onegene_BSA_IFN_RNA['count_IFN'])),
              max(max(onegene_BSA_IFN_RNA['count_BSA']), max(onegene_BSA_IFN_RNA['count_IFN']))]
fig.update_xaxes(range=axis_range)
fig.update_yaxes(range=axis_range)

fig.show()

In [29]:
fig = px.scatter(onegene_BSA_IFN_PRO, x="count_BSA", y="count_IFN", trendline="ols", color="person")

fig.show()

In [30]:
# prompt: force the x and y axis in the graph above to be the same values


fig = px.scatter(onegene_BSA_IFN_PRO, x="count_BSA", y="count_IFN", trendline="ols", color="person")

# Force the x and y axes to have the same range
axis_range = [min(min(onegene_BSA_IFN_PRO['count_BSA']), min(onegene_BSA_IFN_PRO['count_IFN'])),
              max(max(onegene_BSA_IFN_PRO['count_BSA']), max(onegene_BSA_IFN_PRO['count_IFN']))]
fig.update_xaxes(range=axis_range)
fig.update_yaxes(range=axis_range)

fig.show()